In [4]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

model =Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

TRAINING_DIR = "./train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "./test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')


history = model.fit_generator(train_generator,
                              epochs=200,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.
Epoch 1/200
132/132 [==============================] - 42s 316ms/step - loss: 0.5762 - acc: 0.7278 - val_loss: 0.2086 - val_acc: 0.9485
Epoch 2/200
132/132 [==============================] - 43s 324ms/step - loss: 0.3519 - acc: 0.8586 - val_loss: 0.1271 - val_acc: 0.9536
Epoch 3/200
132/132 [==============================] - 42s 315ms/step - loss: 0.2706 - acc: 0.9019 - val_loss: 0.1441 - val_acc: 0.9485
Epoch 4/200
132/132 [==============================] - 41s 312ms/step - loss: 0.2264 - acc: 0.9133 - val_loss: 0.1298 - val_acc: 0.9433
Epoch 5/200
132/132 [==============================] - 42s 315ms/step - loss: 0.2882 - acc: 0.8859 - val_loss: 0.1759 - val_acc: 0.9588
Epoch 6/200
132/132 [==============================] - 41s 311ms/step - loss: 0.2780 - acc: 0.9027 - val_loss: 0.1480 - val_acc: 0.9691
Epoch 7/200
132/132 [==============================] - 42s 319ms/step - loss: 0.2469 - acc: 0.9171 - 

In [2]:
pip install --upgrade Pillow

Requirement already up-to-date: Pillow in /Users/satyamomar/opt/anaconda3/lib/python3.7/site-packages (7.2.0)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
classifier = cv2.CascadeClassifier('/home/shivam/.local/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()